# 必須課題(1) 与えられたコーパスに対する検索の実現
今回は１.京都観光に関する８３件の文書を利用することにした。
まず、kyoto_results_100.jsonから得たBOW集合に対して、ストップワードの除去を実施することにより、検索に適したBOW集合を得る。
nltkに日本語向けストップワードは存在しないため、slothlibが提供している日本語向けストップワードリストを使用した。
ストップワードを除去した結果は次の出力のようなものになる。

In [2731]:
import numpy as np
import gensim
from nltk.corpus import stopwords
import pandas as pd
np.set_printoptions(precision=4)
%precision 3
import json
import urllib.request
import sys

with open("./data/kyoto_results_100.json", "r") as f:
    docs = json.load(f)

    
#nltkには日本語のストップワードリストがないそうなのでslothlibのものを使用します
slothlib_URL = 'http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt'
slothlib_response = urllib.request.urlopen(slothlib_URL)
j_SWtxt = slothlib_response.read().decode('utf-8')
j_SWlist = j_SWtxt.strip()

#ストップワードを除去したコーパスの作成をします
Jdocs = []
for i in range(len(docs)):
    Jdocs.append(docs[i]["bow"].strip().split())
       
Jcorpus = [list(filter(lambda word: word not in j_SWlist, x)) for x in Jdocs]
print(Jcorpus[0])

['定番', '穴場', 'ススメ', '京都', '観光', '...-', 'NAVER', 'まとめ', '関西', '住ん', 'いる', '作者', 'ススメ', '京都', '観光', '名所', 'まとめ', '定番', '穴場', 'スポット', '随時', 'まとめ', 'いき', 'ます', '是非', '京都', '観光', '参考', '下さい']


続いて、ドキュメントに出現する単語とその出現頻度をペアにして辞書化を行う。
今回用いる単語辞書は以下に出力される。

In [2732]:
Jdict = gensim.corpora.Dictionary(Jcorpus)
Jdict.token2id

{'...-': 0,
 'NAVER': 1,
 'いき': 2,
 'いる': 3,
 'ます': 4,
 'まとめ': 5,
 'ススメ': 6,
 'スポット': 7,
 '下さい': 8,
 '京都': 9,
 '住ん': 10,
 '作者': 11,
 '参考': 12,
 '名所': 13,
 '定番': 14,
 '是非': 15,
 '穴場': 16,
 '観光': 17,
 '関西': 18,
 '随時': 19,
 '-': 20,
 '10': 21,
 '1740': 22,
 'TOP': 23,
 'net': 24,
 'おすすめ': 25,
 'なら': 26,
 'らん': 27,
 'セレクト': 28,
 'ランキング': 29,
 '人気': 30,
 '伏見': 31,
 '口コミ': 32,
 '周辺': 33,
 '大社': 34,
 '探す': 35,
 '清水寺': 36,
 '稲荷': 37,
 '紹介': 38,
 'でき': 39,
 'イベント': 40,
 'エリア': 41,
 'ガイド': 42,
 'サイト': 43,
 'ジャンル': 44,
 '公式': 45,
 '季節': 46,
 '幅広く': 47,
 '情報': 48,
 '掲載': 49,
 '桜': 50,
 '検索': 51,
 '温泉': 52,
 '満載': 53,
 '紅葉': 54,
 '美術館': 55,
 '連盟': 56,
 '開花': 57,
 '～': 58,
 '～。': 59,
 '|': 60,
 'ご利益': 61,
 'です': 62,
 'コース': 63,
 'テーマ': 64,
 '仏像': 65,
 '庭園': 66,
 '楽しむ': 67,
 '歳時記': 68,
 '解説': 69,
 '...。': 70,
 'JR': 71,
 'お寺': 72,
 'なる': 73,
 'インタビュー': 74,
 '土産': 75,
 '旅行': 76,
 '東海': 77,
 '案内': 78,
 '楽しく': 79,
 '特集': 80,
 '神社': 81,
 '祭り': 82,
 '行こ': 83,
 '記事': 84,
 '風景': 85,
 '食事': 86,
 '...': 87,
 

続いて、先程用意したBoW集合に対して、辞書を用いてそれぞれの単語を単語IDに変換する。
変換した結果は以下の出力の通りである。

In [2733]:
#単語をIDへ変換
id_Jcorpus = [Jdict.doc2bow(Jdoc) for Jdoc in Jcorpus]
id_Jcorpus[82]

[(0, 1),
 (1, 1),
 (4, 1),
 (5, 1),
 (7, 1),
 (9, 4),
 (14, 2),
 (16, 2),
 (17, 2),
 (25, 1),
 (30, 3),
 (38, 1),
 (41, 1),
 (62, 1),
 (87, 1),
 (161, 1),
 (166, 1),
 (641, 1),
 (829, 1),
 (837, 2),
 (1026, 1)]

単語IDと出現頻度のペアに変換されたBOW集合を、TFIDFで重み付けられた特徴ベクトルに変換する。
今回は、与えられた例と同じく、文書長に対する正規化を行わない。

In [2734]:
#TFIDFの特徴ベクトルを得ます
Jtfidf_model = gensim.models.TfidfModel(id_Jcorpus, normalize=False)
tfidf_Jcorpus = Jtfidf_model[id_Jcorpus]
tfidf_Jcorpus[0]

[(0, 3.790),
 (1, 5.375),
 (2, 5.375),
 (3, 3.568),
 (4, 1.053),
 (5, 11.370),
 (6, 10.750),
 (7, 1.246),
 (8, 6.375),
 (9, 0.106),
 (10, 6.375),
 (11, 6.375),
 (12, 6.375),
 (13, 2.053),
 (14, 7.135),
 (15, 6.375),
 (16, 6.750),
 (17, 0.106),
 (18, 4.790),
 (19, 6.375)]

上記を用いてドキュメントとクエリのコサイン類似度を計算する関数JQsimを定義する。
上述のTFIDFで重み付けられたベクトルをnumpyのベクトルで取得し、クエリに対しても同様の処理を行い、各ドキュメントごとのベクトルとクエリのベクトルとのコサイン類似度を計算し、そのリストを出力する関数である。引数にクエリ、TDIDFベクトル化したドキュメント集合、ドキュメントに対するTFIDFモデル、ドキュメント全体に対して作成した辞書を受け取る。

In [2735]:
def JQsim(query, tfidf_corpus, tfidf_model, dictionary):
    #クエリのtfidfベクトルとドキュメントのtfidfベクトルをそれぞれ作成する
    tfidf_query = tfidf_model[dictionary.doc2bow(query)]
    query_vector = gensim.matutils.corpus2dense([tfidf_query], len(dictionary)).T[0]
    tfidf_vectors = gensim.matutils.corpus2dense(tfidf_corpus, len(dictionary)).T
    #コサイン類似度はゼロベクトルに対して計算出来ないため、エラーを出力する
    if (all(elm == 0 for elm in query_vector)):
        print("全文書に含まれていない単語で検索しているため、ランキングを作成出来ません")
        sys.exit()
    #各ドキュメント毎にクエリに対するコサイン類似度を計算した値を入れたリストを作成する
    sim_list = []
    for i, tfidf_vec in enumerate(tfidf_vectors): #enumerate化注意
        sim_list.append(1.0 - cosine(query_vector, tfidf_vec))
    #上記リストを返り値とします
    return sim_list

続いて、上述の関数を用いて、
各ドキュメント毎のクエリに対するコサイン類似度を計算した値を入れたリストをソートし、
トップ１０のドキュメントを発見する。クエリは観光で用いられそうなクエリ４つを用意した。

In [2736]:
querylist = [{'京都', "泊まる", "おすすめ"}, {"ランチ", "美味しい"}, {"景色", "電車"}, ("行き方", "金閣寺")]

#上記関数を用いて、クエリ1の各ドキュメントに対するコサイン類似度のリストを取得します
sim_list = JQsim(querylist[0], tfidf_Jcorpus, Jtfidf_model, Jdict)

コサイン類似度順にドキュメント番号をソートする関数を作り、それでランキングを行う。

In [2737]:
def ranking (sim_list):
    sim_dic = {}
    for i, score in enumerate(sim_list):
        sim_dic[i] = score
    rank = sorted(sim_dic.items(), key=lambda x: x[1])
    rank.reverse()
    return(rank)

#それぞれのクエリに対して上記の関数を実行し、クエリとのコサイン類似度が高いドキュメント　トップ１０を出力する
rank = ranking(sim_list)
print("query: ",querylist[0])
for i in range(9):
    print("document_id:"+str(rank[i][0])+"\n document_summary[" + docs[rank[i][0]]["summary"] + "]")

query:  {'おすすめ', '京都', '泊まる'}
document_id:60
 document_summary[京都観光おすすめの穴場スポットや人気の伝統行事などを案内しています。国際観光都市である京都は四季折々の風景や食・文化・名所など楽しめます。京都各地おすすめの観光コースや市バス、観光バス情報。]
document_id:3
 document_summary[京都観光をじっくり楽しむためのサイトです。おすすめコースをエリア別やテーマ別にガイドしています。庭園や仏像、ご利益さん、歳時記なども解説しています。]
document_id:30
 document_summary[京都観光のおすすめをどこよりも早く紹介します。京都を旅するならここを見てから！最新情報からおすすめのホテル・旅館・グルメ・寺社仏閣までをご紹介します！]
document_id:7
 document_summary[定番の京都観光コースから、京都人でも知らないようなとっておきのおすすめ観光コースまで、いろいろなテーマの京都観光コースをご紹介しています。それぞれの立ち寄りスポットの詳細な情報も掲載しております。]
document_id:35
 document_summary[もっと京都が好きになる！京都通もおすすめする京都観光完全ガイド。京都の魅力がたっぷり詰まったグルメ・神社・寺院・和菓子・歴史ガイドをあますことなくランキングで紹介します！]
document_id:10
 document_summary[京都の観光スポット情報、イベント・行事スケジュール、おすすめ観光コースのご紹介、全て無料で使える京都の写真ギャラリーを中心に、さまざまな京都観光の情報が満載の、京都観光情報ポータルサイトです。観光スポットの情報 ...]
document_id:64
 document_summary[京都嵐山観光おすすめ！人気散策コースから周辺グルメまでご紹介！観光 京都の代表的な観光地で桜や紅葉の名所にもなっている嵐山。一日過ごしても飽きないくらい観たり食べたり買い物したりする人気スポットがいっぱいです。]
document_id:63
 document_summary[京都府京都市の観光情報についてまとめました。全国のみならず海外からも多くの観光客が訪れる京都。人気の定番ど

クエリ２,クエリ３,クエリ4についても実行する。

In [2738]:
#クエリ4検索結果
sim_list = JQsim(querylist[1], tfidf_Jcorpus, Jtfidf_model, Jdict)
rank = ranking(sim_list)
print("query: ",querylist[1])
for i in range(9):
    print("document_id:"+str(rank[i][0])+"\n document_summary[" + docs[rank[i][0]]["summary"] + "]")

query:  {'ランチ', '美味しい'}
document_id:80
 document_summary[スバコ・JR京都伊勢丹の3階にある「中村藤吉」は、京都・宇治の老舗茶舗が営むお店です。甘味処ですが、ランチタイムにおいしい茶蕎麦が食べられると人気のお店です。食後には「ほうじ茶ソフトクリーム」もおすすめです。]
document_id:69
 document_summary[【京都】天橋立の魂を揺さぶる人気＆おすすめ観光スポット24選！ 京都府宮津市にある天橋立は日本三景のひとつとして人気の観光地です。アオリイカをはじめとする美味しい海産物があり、ランチをする場所にも困りません。]
document_id:82
 document_summary[京都 祇園は、京都の中でも特に人気の高いエリアです。その魅力を人気＆定番スポットはもちろん穴場も紹介しながらお伝えしたいと思います。限られた時間の中で京都祇園を...]
document_id:81
 document_summary[見所の多い京都観光の中でも、特に人気が高い嵐山・嵯峨野エリア。この辺りは、古くから貴族や文人たちが都を逃れ「わび住まい」をした風流の地だけあり、周囲の山々が紅葉に染まる晩秋ともなれば、その風情もひとしおです。]
document_id:79
 document_summary[清水寺に近くて観光に利用させていただきました。 0歳と2歳の子供がいたのですが、一軒家を貸しきりだったので、そこまで子供の騒音を気にせずにすみました。 チェックアウト後も荷物を預かってくれ… 2016-08-27 16:03:39投稿 つづ ...]
document_id:78
 document_summary[観光を楽しむならエースJTB『旅の過ごし方BOOK』付きプランがおすすめ！ これを使えば、現地でその都度お金を使うよりもお得にサービスを利用できて、とっても便利！ 例えば・・・ 観光に疲れたら観光名所の近くのエースJTBのお休み ...]
document_id:77
 document_summary[JR東海の京都観光情報【そうだ 京都、行こう。公式サイト】。京都旅行の便利帳、散策コース紹介ページ。京都の風景、お寺や神社、桜、紅葉、イベント、お祭り、お食事やお土産、宿泊などの情報をご案内。

In [2739]:
#クエリ4検索結果
sim_list = JQsim(querylist[2], tfidf_Jcorpus, Jtfidf_model, Jdict)
rank = ranking(sim_list)
print("query: ",querylist[2])
for i in range(9):
    print("document_id:"+str(rank[i][0])+"\n document_summary[" + docs[rank[i][0]]["summary"] + "]")

query:  {'電車', '景色'}
document_id:67
 document_summary[京都の繁華街・四条河原町は電車・バスが京都市内でも最も豊富であるがために、どの交通手段を選択するべきか迷うところです。今回は、四条河原町から各観光名所・人気スポットへのアクセス方法をまとめました。]
document_id:19
 document_summary[【定番から穴場まで】おススメの京都観光スポット【56… USJ攻略法 〜USJを120%楽しむ方法まとめ スーツケース・キャリーケースの選び方 日本国内の死ぬまでに一度は行きたい観光名所 PRまとめ そんなのあんの!?ランナーたちも愛用 ...]
document_id:82
 document_summary[京都 祇園は、京都の中でも特に人気の高いエリアです。その魅力を人気＆定番スポットはもちろん穴場も紹介しながらお伝えしたいと思います。限られた時間の中で京都祇園を...]
document_id:81
 document_summary[見所の多い京都観光の中でも、特に人気が高い嵐山・嵯峨野エリア。この辺りは、古くから貴族や文人たちが都を逃れ「わび住まい」をした風流の地だけあり、周囲の山々が紅葉に染まる晩秋ともなれば、その風情もひとしおです。]
document_id:80
 document_summary[スバコ・JR京都伊勢丹の3階にある「中村藤吉」は、京都・宇治の老舗茶舗が営むお店です。甘味処ですが、ランチタイムにおいしい茶蕎麦が食べられると人気のお店です。食後には「ほうじ茶ソフトクリーム」もおすすめです。]
document_id:79
 document_summary[清水寺に近くて観光に利用させていただきました。 0歳と2歳の子供がいたのですが、一軒家を貸しきりだったので、そこまで子供の騒音を気にせずにすみました。 チェックアウト後も荷物を預かってくれ… 2016-08-27 16:03:39投稿 つづ ...]
document_id:78
 document_summary[観光を楽しむならエースJTB『旅の過ごし方BOOK』付きプランがおすすめ！ これを使えば、現地でその都度お金を使うよりもお得にサービスを利用できて、とっても便利！ 例えば・・・ 観光に疲れたら観光名所の

In [2740]:
#クエリ4検索結果
sim_list = JQsim(querylist[3], tfidf_Jcorpus, Jtfidf_model, Jdict)
rank = ranking(sim_list)
print("query: ",querylist[3])
for i in range(9):
    print("document_id:"+str(rank[i][0])+"\n document_summary[" + docs[rank[i][0]]["summary"] + "]")

query:  ('行き方', '金閣寺')
document_id:71
 document_summary[金閣寺は京都を代表する観光地のひとつ。銀閣寺・嵐山・清水寺などとならび、五本の指に入る人気スポットです。ところが京都観光は移動が大変。交通手段も複雑です。そこで今回は、金閣寺から各観光名所・人気スポットへの ...]
document_id:17
 document_summary[京都観光の楽しみ方 京都観光といえば、金閣寺・銀閣寺の室町時代の将軍が建てた寺や 清水寺、高台寺、八坂神社などの東山エリアのお寺めぐりを楽しんだり 春には鮮やかな桜の花見、秋には美しい紅葉を鑑賞する京都観光や]
document_id:46
 document_summary[京都の観光名所は広範囲に散らばっているため、慣れていないと移動だけでひと苦労。限られた時間の中で効率よく見て回りたいなら、定期観光バスや観光タクシーが便利です。清水寺や金閣寺など人気の観光名所を巡る王道コースを ...]
document_id:24
 document_summary[長きにわたって日本の首都として栄えた古都京都は、市街地に清水寺や金閣寺（鹿苑寺）、平安神宮や北野天満宮といった全国に名]
document_id:20
 document_summary[春夏の京都観光スポットを巡るおすすめバスツアー予約。バスガイドが京都のお寺や名所を案内する定期観光バスで行くJTBの日帰りバスツアー！比叡山や大原三千院、金閣寺、銀閣寺など京都観光に迷ったらこちのページをチェック。]
document_id:82
 document_summary[京都 祇園は、京都の中でも特に人気の高いエリアです。その魅力を人気＆定番スポットはもちろん穴場も紹介しながらお伝えしたいと思います。限られた時間の中で京都祇園を...]
document_id:81
 document_summary[見所の多い京都観光の中でも、特に人気が高い嵐山・嵯峨野エリア。この辺りは、古くから貴族や文人たちが都を逃れ「わび住まい」をした風流の地だけあり、周囲の山々が紅葉に染まる晩秋ともなれば、その風情もひとしおです。]
document_id:80
 document_summary[スバコ・JR京都伊勢丹の3階にある「中村藤吉」は、京

## 結果に対する考察
コサイン類似度を利用するため、ドキュメント集合から作成した単語辞書の中に存在しない単語でクエリを投げられた場合、類似度が計算できない問題点が存在している。そこを改善するために、word2vecを利用して単語同士の類似度を計算したり、より広い単語辞書を用いるような手段が考えられる。また、複数の単語でクエリを作成した際、一方の単語のみ辞書に入っていた場合、そちらの単語でのみ類似度を計算しランキングしてしまうため、ユーザーにとって不可解な偏った結果を返してしまう問題も存在している。こちらも、上記と同様の手法で改善すると思われる。